Very WIP notebooks to help finish up the project.

In [58]:
import os
os.chdir("/home/smestern/patchotda/")
import streamlit as st
import pandas as pd
import numpy as np
from concurrent import futures
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from hiclass import LocalClassifierPerNode, LocalClassifierPerParentNode, LocalClassifierPerLevel
from sklearn.base import BaseEstimator, ClassifierMixin
#import XGBoost as xgb
#other
import altair as alt
import time
import zipfile
from utils import MMS_DATA, USER_DATA, EXAMPLE_DATA_, REF_DATA_, VISp_MET_nodes, VISp_T_nodes, filter_MMS, find_outlier_idxs, param_grid_from_dict, select_by_col, not_select_by_col
import os
from patchOTDA.external import skada
from patchOTDA.domainAdapt import PatchClampOTDA
from functools import partial
import ot.da
from ot.backend import get_backend
import umap
from streamlit_tree_select import tree_select
import xgboost as xgb
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE
from streamlit_app import CLASS_MODELS, HICLASS_METHOD

labels="VISp_T"
hiclass_method = "LocalClassifierPerNode"
class_model = "Random Forest"


In [59]:
ref_data = MMS_DATA['VISp_Viewer']
ref_data_ephys, ref_data_meta = ref_data['ephys'], ref_data['meta']
Xs, Ys = ref_data_ephys, ref_data_meta[[labels+"_1_en", labels+"_2_en", labels+"_3_en"]]
Ys = Ys.to_numpy()
print(Ys)

[[ 5 38 49]
 [ 4 33 42]
 [ 4 20 24]
 ...
 [ 1 14 16]
 [ 1 14 16]
 [ 1 10 11]]


In [60]:
Xs_train, Xs_test, Ys_train, Ys_test = train_test_split(Xs, Ys, test_size=0.2, random_state=42)
#fix the imbalance in the train


In [61]:
impute_and_scale = Pipeline([('imputer', KNNImputer()), ('scaler', StandardScaler())])
Xs_train = impute_and_scale.fit_transform(Xs_train)
Xs_test = impute_and_scale.transform(Xs_test)
Xs = impute_and_scale.transform(Xs)
print(Xs_train.shape)
smote_enn = SMOTEENN(random_state=42, smote=SMOTE(k_neighbors=1,random_state=42))
Xs_train, Ys_train = smote_enn.fit_resample(Xs_train, Ys_train[:,-1])
print(Xs_train.shape)

(3408, 29)
(10754, 29)


In [55]:

CLASS_MODELS[class_model]['params']
grid_params = param_grid_from_dict(CLASS_MODELS[class_model]['params'])

In [56]:

class dummy_cv_grid(BaseEstimator, ClassifierMixin): #this is a hack to get around the fact that the gridsearchcv does not like the LocalClassifierPerNode
    def __init__(self, **kwargs):
        self.hiclass = HICLASS_METHOD[hiclass_method](CLASS_MODELS[class_model]['model'](**kwargs))
    def set_params(self, **params):
        self.hiclass = HICLASS_METHOD[hiclass_method](CLASS_MODELS[class_model]['model'](**params))
        return self
    def fit(self, X, y):
        self.hiclass.fit(X, y)
        return self
    
    def predict(self, X):
        return self.hiclass.predict(X)

    def score(self, X, y):
        self.hiclass.fit(X, y)
        y_pred = self.hiclass.predict(X)
        score = []
        for level in np.arange(y.shape[1]):
            
            score.append(accuracy_score(y[:,level].astype(np.int32), y_pred[:,level].astype(np.int32)))
        return np.mean(score)



model = HICLASS_METHOD[hiclass_method](HistGradientBoostingClassifier(),n_jobs=-1)






In [57]:
#split x_train and y_train into 2/3 training and 1/3 validation

def error_func(Xs_train, Ys_train, params):
    #grid search
    himethod = params.pop('model')
    #quick 5 fold cross validation
    error = 0
    for x in range(5):
        Xs_train_temp, Xs_val_temp, Ys_train_temp, Ys_val_temp = train_test_split(Xs_train, Ys_train, test_size=0.33, random_state=x, shuffle=True)
        model = himethod(**params)
        model.fit(Xs_train_temp, Ys_train_temp[:,-1])
        Ys_pred = model.predict(Xs_val_temp)
        error += balanced_accuracy_score(Ys_val_temp[:,-1].astype(np.int32), Ys_pred.astype(np.int32))
    print(error/5)
    return 1-(error/5)


#make a nevergrad optimizer
import nevergrad as ng

# param_space = ng.p.Dict(**{'HICLASS_METHOD': ng.p.Choice(['LocalClassifierPerNode', 'LocalClassifierPerParentNode']),
#                         #'n_estimators': ng.p.Log(lower=10, upper=1000).set_integer_casting(), 
#                          #'min_samples_split': ng.p.Scalar(lower=0.001, upper=1), 
#                          'min_samples_leaf': ng.p.Scalar(lower=0.001, upper=0.5),
#                          'max_depth': ng.p.TransitionChoice(choices=[*np.arange(1, 100).tolist(), None]), 
#                          'max_features': ng.p.Scalar(lower=0.001, upper=0.5),
#                          'max_leaf_nodes' 
#                         # 'min_impurity_decrease': ng.p.Log(lower=1e-4, upper=1),})
# })


# param_space = ng.p.Dict(**{'HICLASS_METHOD': ng.p.Choice(['LocalClassifierPerNode', 'LocalClassifierPerParentNode', 'LocalClassifierPerLevel']),
#                         'learning_rate': ng.p.Log(lower=0.001, upper=1),
#                         'l2_regularization': ng.p.Log(lower=1e-4, upper=500),
#                          'min_samples_leaf': ng.p.Scalar(lower=1, upper=100).set_integer_casting(),
#                          'max_depth': ng.p.TransitionChoice(choices=[*np.arange(1, 100).tolist(), None]), 
#                          'max_features': ng.p.Scalar(lower=0.001, upper=0.5),
#                          'max_leaf_nodes': ng.p.TransitionChoice(choices=[*np.arange(2, 100).tolist(), None]),
#                          'class_weight': ng.p.Choice(['balanced', None]),   
# })

param_space = ng.p.Choice([ng.p.Dict(**{'model': HistGradientBoostingClassifier,
                                                'learning_rate': ng.p.Log(lower=0.001, upper=1),
                                                'l2_regularization': ng.p.Log(lower=1e-2, upper=500),
                                                'min_samples_leaf': ng.p.Scalar(lower=1, upper=100).set_integer_casting(),
                                                    
                                                'max_depth': ng.p.TransitionChoice(choices=[*np.arange(1, 100).tolist(), None]),
                                                'max_features': ng.p.Scalar(lower=0.001, upper=1),
                                                'max_leaf_nodes': ng.p.TransitionChoice(choices=[*np.arange(2, 100).tolist(), None]),
                                                'class_weight': ng.p.Choice(['balanced', None]),
                                                }), 
                           ng.p.Dict(**{'model': RandomForestClassifier,
                                                'n_estimators': ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
                                                'min_samples_split': ng.p.Scalar(lower=0.001, upper=1),
                                                'min_samples_leaf': ng.p.Scalar(lower=0.001, upper=0.5),
                                                'min_impurity_decrease': ng.p.Log(lower=1e-4, upper=1),
                                                'max_depth': ng.p.TransitionChoice(choices=[*np.arange(1, 100).tolist(), None]),
                                                'max_features': ng.p.Scalar(lower=0.001, upper=0.5),
                                                }),
                            ng.p.Dict(**{'model': AdaBoostClassifier,
                                                'n_estimators': ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
                                                'learning_rate': ng.p.Log(lower=0.001, upper=1),
                                                }),
                            ng.p.Dict(**{'model': ExtraTreesClassifier,
                                                'n_estimators': ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
                                                'min_samples_split': ng.p.Scalar(lower=0.001, upper=1),
                                                'min_samples_leaf': ng.p.Scalar(lower=0.001, upper=0.5),
                                                'min_impurity_decrease': ng.p.Log(lower=1e-4, upper=1),
                                                'class_weight': ng.p.Choice(['balanced', None]),}),
                                                ])




optimizer = ng.optimizers.Portfolio(parametrization=param_space, budget=1000, num_workers=24,)

#run the optimizer
with futures.ThreadPoolExecutor(max_workers=24) as executor:
    recommendation = optimizer.minimize(partial(error_func, Xs_train, Ys_train),  executor=executor, batch_mode=False,verbosity=2)

Launching 24 jobs with new suggestions


KeyboardInterrupt: 

In [62]:
best_val = recommendation.value
himethod = best_val.pop('model')
print(best_val)
recommendation

NameError: name 'recommendation' is not defined

In [72]:
#reinit the model and train on  the full dataset
#print(himethod)
model = HICLASS_METHOD['LocalClassifierPerNode'](RandomForestClassifier(max_depth=15),n_jobs=-1)
model.fit(Xs_train, Ys_train.astype(np.int32))

model2 = HistGradientBoostingClassifier()
model2.fit(Xs_train, Ys_train[:].astype(np.int32))


HistGradientBoostingClassifier()

In [69]:
Ys_pred = model.predict(Xs_test)
Ys_pred_train = model.predict(Xs_train)

Ys_pred2 = model2.predict(Xs_test)
#test the 3 levels of the model
# dict_results = {"Train":[], "Test":[]}
# for level in [0,1,2]:
#     dict_results[f"Train"].append(accuracy_score(Ys_train[:,level].astype(np.int32), Ys_pred_train[:,level].astype(np.int32)))
#     dict_results[f"Test"].append(accuracy_score(Ys_test[:,level].astype(np.int32), Ys_pred[:,level].astype(np.int32)))


In [70]:
print(balanced_accuracy_score(Ys_pred_train.astype(np.int32), Ys_train.astype(np.int32)))
print(balanced_accuracy_score(Ys_pred.astype(np.int32), Ys_test[:,-1].astype(np.int32)))

0.962395797371976
0.29020505612343606


/home/smestern/miniconda3/envs/patchotda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [71]:
print(dict_results)

NameError: name 'dict_results' is not defined

In [ ]:
print(balanced_accuracy_score(Ys_pred2.astype(np.int32), Ys_test[:,-1].astype(np.int32)))

0.15319974382380155


In [ ]:
label_encoder = MMS_DATA['VISp_Viewer'].keys()
print(label_encoder)

dict_keys(['ephys', 'meta', 'VISp_MET_LE_1', 'VISp_MET_LE_2', 'VISp_MET_LE_3', 'VISp_T_LE_1', 'VISp_T_LE_2', 'VISp_T_LE_3'])


In [ ]:
#create a confusion matrix for each level
from sklearn.metrics import ConfusionMatrixDisplay
confusion_matrices = []
for level in [0,1,2]:
    label_encoder = MMS_DATA['VISp_Viewer'][f'VISp_T_LE_{level+1}']
    confusion_matrices.append(confusion_matrix(label_encoder.inverse_transform(Ys_test[:,level].astype(np.int32)), label_encoder.inverse_transform(Ys_pred[:,level].astype(np.int32)), labels=label_encoder.classes_))
    #save as a dataframe csv
    pd.DataFrame(confusion_matrices[-1], columns=label_encoder.classes_, index=label_encoder.classes_).to_csv(f"confusion_matrix_{level}.csv")


    


